In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Python Spark SQL basic example") \
  .config("spark.sql.config.option", "some-value") \
  .getOrCreate()

## 7. Aggregation
- 구매 이력 데이터를 사용해 파티션을 훨씬 적은 수로 분할할 수 있도록 리파티셔닝
- 빠르게 접근할 수 있도록 캐싱

In [3]:
df = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("./FileStore/tables/data/retail-data/all/*.csv") \
  .coalesce(5)
  
df.cache()
df.createOrReplaceTempView("dTable")

In [4]:
display(df.limit(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [5]:
df.count()

Out[6]: 541909

## 7.1 집계함수
- org.apache.spark.sql.functions 패키지

### 7.1.1 count

In [8]:
from pyspark.sql.functions import col, count

df.select(count("StockCode")).show() # null 로우는 제외
df.select(count("*")).show()     # null 로우는 포함

+----------------+
count(StockCode)|
+----------------+
 541909|
+----------------+

+--------+
count(1)|
+--------+
 541909|
+--------+

7.1.2 countDistinct
- 고유레코드 수

In [10]:
from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct("StockCode", 0.1)).show() # 0.1은 최대 오류 추정률
df.select(approx_count_distinct("StockCode", 0.01)).show()

+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 3364|
+--------------------------------+

+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 4079|
+--------------------------------+

### 7.1.4 first와 last

In [12]:
from pyspark.sql.functions import first, last

display(df.select(first("StockCode"), last("StockCode")).show(1))

+-----------------------+----------------------+
first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
 85123A| 22138|
+-----------------------+----------------------+

### 7.1.5 min과 max
- 문자열도 동작이 됨

In [14]:
from pyspark.sql.functions import min, max

df.select(min("Quantity"), max("Quantity")).show(1)
df.select(min("Description"), max("Description")).show(1)

+-------------+-------------+
min(Quantity)|max(Quantity)|
+-------------+-------------+
 -80995| 80995|
+-------------+-------------+

+--------------------+-----------------+
 min(Description)| max(Description)|
+--------------------+-----------------+
 4 PURPLE FLOCK D...|wrongly sold sets|
+--------------------+-----------------+

### 7.1.6 sum

In [16]:
from pyspark.sql.functions import sum

display(df.select(sum("Quantity")).limit(1))

sum(Quantity)
5176450


### 7.1.7 sumDistinct

In [18]:
from pyspark.sql.functions import sumDistinct

display(df.select(sumDistinct("Quantity")).limit(1)) # 고유값을 합산

sum(DISTINCT Quantity)
29310


### 7.1.8 avg
- avg, mean 함수로 평균을 구함

In [20]:
from pyspark.sql.functions import sum, count, avg, expr

display(df.select(
    count("Quantity").alias("total_transcations"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_transcations"),    
).selectExpr(
    "total_purchases / total_transcations",
    "avg_purchases",
    "mean_transcations").limit(5)
)

(total_purchases / total_transcations),avg_purchases,mean_transcations
9.55224954743324,9.55224954743324,9.55224954743324


### 7.1.9 분산과 표준편차
- 표본표준분산 및 편차 : variance, stddev
- 모표준분산 및 편차 : var_pop, stddev_pop

In [22]:
from pyspark.sql.functions import variance, stddev
from pyspark.sql.functions import var_samp, stddev_samp
from pyspark.sql.functions import var_pop, stddev_pop

display(df.select(variance("Quantity"), stddev("Quantity"),
        var_samp("Quantity"), stddev_samp("Quantity"),
        var_pop("Quantity"), stddev_pop("Quantity")).limit(3)
)

var_samp(Quantity),stddev_samp(Quantity),var_samp(Quantity),stddev_samp(Quantity),var_pop(Quantity),stddev_pop(Quantity)
47559.39140929886,218.08115785023443,47559.39140929886,218.08115785023443,47559.30364660917,218.0809566344782


In [23]:
display(spark.createDataFrame(df.select("*").take(1))
  .select(variance("Quantity"), stddev("Quantity"),
          var_samp("Quantity"), stddev_samp("Quantity"),
          var_pop("Quantity"), stddev_pop("Quantity")))

var_samp(Quantity),stddev_samp(Quantity),var_samp(Quantity),stddev_samp(Quantity),var_pop(Quantity),stddev_pop(Quantity)
NaN,NaN,NaN,NaN,0.0,0.0


### 7.1.10 비대칭도와 첨도

In [25]:
from pyspark.sql.functions import skewness, kurtosis

display(df.select(skewness("Quantity"), kurtosis("Quantity")))

skewness(Quantity),kurtosis(Quantity)
-0.26407557610528065,119768.05495536947


### 7.1.11 공분산과 상관관계
- 표본공분산(cover_samp), 모공분산(cover_pop)

In [27]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

display(df.select(corr("InvoiceNo", "Quantity"), covar_pop("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity")))

"corr(InvoiceNo, Quantity)","covar_pop(InvoiceNo, Quantity)","covar_samp(InvoiceNo, Quantity)"
4.912186085637207E-4,1052.7260778746647,1052.728054390769


In [28]:
from pyspark.sql.functions import collect_list, collect_set, size

display(df.select(collect_list("Country"), collect_set("Country")).show())

+---------------------+--------------------+
collect_list(Country)|collect_set(Country)|
+---------------------+--------------------+
 [United Kingdom, ...|[Portugal, Italy,...|
+---------------------+--------------------+

In [29]:
display(df.select(size(collect_list("Country")), size(collect_set("Country"))).show()) # 각 컬럼의 복합데이터 사이즈

+---------------------------+--------------------------+
size(collect_list(Country))|size(collect_set(Country))|
+---------------------------+--------------------------+
 541909| 38|
+---------------------------+--------------------------+

In [30]:
from pyspark.sql.functions import countDistinct

display(df.select(countDistinct("Country")).show()) # 중복없이 카운트

+-----------------------+
count(DISTINCT Country)|
+-----------------------+
 38|
+-----------------------+

## 7.2 그룹화
- 하나 이상의 컬럼을 그룹화하여 RelationalGroupedDataset 변환
- 집계 연산을 수행하는 구 번째 단계에서는 DataFrame이 반환됨

In [32]:
display(df.groupBy("InvoiceNo", "CustomerID").count())

InvoiceNo,CustomerID,count
536846,14573,76
537026,12395,12
537883,14437,5
538068,17978,12
538279,14952,7
538800,16458,10
538942,17346,12
C539947,13854,1
540096,13253,16
540530,14755,27


### 7.2.1 표현식을 이용한 그룹화

In [34]:
from pyspark.sql.functions import count

display(df.groupBy("InvoiceNo", "CustomerId").agg(
    count("Quantity").alias("guan"),
    expr("count(Quantity)")).limit(20))

InvoiceNo,CustomerId,guan,count(Quantity)
536846,14573,76,76
537026,12395,12,12
537883,14437,5,5
538068,17978,12,12
538279,14952,7,7
538800,16458,10,10
538942,17346,12,12
C539947,13854,1,1
540096,13253,16,16
540530,14755,27,27


### 7.2.2 맵을 이용한 그룹화
- 파이썬의 딕셔너리 데이터 타입을 활용하여 집계함수의 표현이 가능함

In [36]:
display(df.groupBy("InvoiceNo").agg({"Quantity": "avg", "Quantity": "stddev_pop"}).limit(20)) # 키값이 충돌하는 것을 볼 수 있음

InvoiceNo,stddev_pop(Quantity)
536596,1.1180339887498947
536938,20.698023172885524
537252,0.0
537691,5.597097462078001
538041,0.0
538184,8.142590198943392
538517,2.3946659604837897
538879,11.811070444356483
539275,12.806248474865697
539630,10.225241100118645


### 7.3 윈도우 함수
- group-by 함수를 사용하면 모든 로우 레코드가 단일 그룹으로만 이동하지만, 윈도우 함수는 하나 이상의 프레임에 할당될 수 있음
- 가장 흔한 예시로는 하루를 나타내는 값의 롤링 평균을 구하는 예시

In [38]:
from pyspark.sql.functions import col, to_date

dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/dd/yyyy HH:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")
display(dfWithDate.limit(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01


In [39]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window \
  .partitionBy("CustomerID", "date") \
  .orderBy(desc("Quantity")) \
  .rowsBetween(Window.unboundedPreceding, Window.currentRow) # 파티션의 처음부터 현재 로우까지

In [40]:
from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [41]:
from pyspark.sql.functions import dense_rank, rank

purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [42]:
from pyspark.sql.functions import col

display(dfWithDate.where("CustomerID is NOT NULL").orderBy("CustomerID") \
  .select(col("CustomerID"), col("date"), col("Quantity"),
          purchaseRank.alias("quantityRank"), # 중복 순위를 감안
          purchaseDenseRank.alias("quantityDenseRank"),  # 중복 순위를 감안X
          maxPurchaseQuantity.alias("maxDenseRank")).limit(20))

CustomerID,date,Quantity,quantityRank,quantityDenseRank,maxDenseRank
12346,2011-01-18,74215,1,1,74215
12346,2011-01-18,-74215,2,2,74215
12347,2010-12-07,36,1,1,36
12347,2010-12-07,30,2,2,36
12347,2010-12-07,24,3,3,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36


## 7.4 그룹화 셋
- 여러 그룹에 걸쳐 집계를 결합하는 저수준 기능

In [44]:
dfNoNull = dfWithDate.na.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

display(dfNoNull.groupBy(col("stockCode"), col("CustomerID")) \
  .agg(sum("Quantity").alias("sumOfQuantity")) \
  .orderBy(desc("sumOfQuantity")).limit(20))

stockCode,CustomerID,sumOfQuantity
84826,13256,12540
22197,17949,11692
84077,16333,10080
17003,16422,10077
21915,16333,8120
16014,16308,8000
22616,17306,6624
22189,18102,5946
84077,12901,5712
18007,14609,5586


### 7.4.1 롤업
- 다양한 컬럼을 그룹화 키로 설정하면 데이터셋에서 볼 수 있는 실제 조합을 살펴볼 수 있음
- 컬럼을 계층적으로 다룸
- null 값을 가진 로우에서 전체 날짜의 합계를 확인할 수 있음
- 모두 null인 로우는 두 컬럼에 속한 레코드의 전체 합계를 나타냄

In [46]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity")) \
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity") \
  .orderBy("Date")

# null 값을 가진 로우는 전체 날짜의 합계 확인
rolledUpDF.where(expr("Date IS NULL")).limit(1)

# 모두 Null인 로우는 두 컬럼에 속한 레코드의 전체 합계
display(rolledUpDF.where(expr("Country IS NULL")).limit(20))

Date,Country,total_quantity
null,null,4906888
2010-12-01,null,24032
2010-12-02,null,20855
2010-12-03,null,11548
2010-12-05,null,16394
2010-12-06,null,16095
2010-12-07,null,19351
2010-12-08,null,21275
2010-12-09,null,16904
2010-12-10,null,15388


### 7.4.2 큐브
- 계층적이 아닌 모든 차원에 대해 동일한 작업을 수행

In [48]:
rolledUpDF = dfNoNull.cube("Date", "Country").agg(sum("Quantity")) \
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity") \
  .orderBy("Date")

# null값을 가진 로우는 전체 날짜의 합계 확인
display(rolledUpDF.where(expr("Date IS NULL")).show())

+----+--------------------+--------------+
Date| Country|total_quantity|
+----+--------------------+--------------+
null| Norway| 19247|
null| European Community| 497|
null| Denmark| 8188|
null| Czech Republic| 592|
null| Spain| 26824|
null| Channel Islands| 9479|
null| USA| 1034|
null| Singapore| 5234|
null|United Arab Emirates| 982|
null| Finland| 10666|
null| Lebanon| 386|
null| Cyprus| 6317|
null| Germany| 117448|
null| RSA| 352|
null| Unspecified| 1789|
null| null| 4906888|
null| Australia| 83653|
null| Portugal| 16044|
null| Japan| 25218|
null| Italy| 7999|
+----+--------------------+--------------+
only showing top 20 rows

In [49]:
# 모두 null인 로우는 두 컬럼에 속한 레코드의 전체 합계
display(rolledUpDF.where(expr("Country IS NULL")).limit(20))

Date,Country,total_quantity
null,null,4906888
2010-12-01,null,24032
2010-12-02,null,20855
2010-12-03,null,11548
2010-12-05,null,16394
2010-12-06,null,16095
2010-12-07,null,19351
2010-12-08,null,21275
2010-12-09,null,16904
2010-12-10,null,15388


### 7.4.3 그룹화 메타 데이터
- 결과 데이터셋의 집계 수준을 명시하는 컬럼을 제공

~~~
3 : 가장 높은 계층의 집계 결과에서 나타남
2 : 개별 재고 코드의 모든 집계 결과에서 나타남
1 : 구매한 물품에 관계없이 customerID를 기잔으로 총 수량을 제공
0 : CustomerID와 stockCode별 조합에 따라 총 수량을 제공
~~~

In [51]:
from pyspark.sql.functions import grouping_id, sum, expr

display(dfNoNull.cube("customerID", "stockcode") \
  .agg(grouping_id(), sum("Quantity")) \
  .orderBy(col("grouping_id()").desc()) \
  .where("grouping_id() = 3").limit(3))

customerID,stockcode,grouping_id(),sum(Quantity)
null,null,3,4906888


In [52]:
display(dfNoNull.cube("customerID", "stockcode") \
  .agg(grouping_id(), sum("Quantity")) \
  .orderBy(col("grouping_id()").desc()) \
  .where("grouping_id() = 2").limit(3))

customerID,stockcode,grouping_id(),sum(Quantity)
null,21676,2,211
null,90059E,2,4
null,22295,2,2725


In [53]:
display(dfNoNull.cube("customerID", "stockcode") \
  .agg(grouping_id(), sum("Quantity")) \
  .orderBy(col("grouping_id()").desc()) \
  .where("grouping_id() = 1").limit(3))

customerID,stockcode,grouping_id(),sum(Quantity)
14506,null,1,730
14850,null,1,285
14896,null,1,279


In [54]:
display(dfNoNull.cube("customerID", "stockcode") \
  .agg(grouping_id(), sum("Quantity")) \
  .orderBy(col("grouping_id()").desc()) \
  .where("grouping_id() = 0").limit(3))

customerID,stockcode,grouping_id(),sum(Quantity)
13047,22912,0,3
14688,21212,0,125
13705,22128,0,12


### 7.4.4 피벗
- 로우를 컬럼으로 변환할 수 있음

In [56]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()
# pivoted columns0
display(pivoted.where("date > '2011-12-05'").select("date", "USA_sum(CAST(Quantity AS BIGINT))").show())

+----------+---------------------------------+
 date|USA_sum(CAST(Quantity AS BIGINT))|
+----------+---------------------------------+
2011-12-06| null|
2011-12-09| null|
2011-12-08| -196|
2011-12-07| null|
+----------+---------------------------------+

## 7.5 사용자 정의 집계 함수
- User Defined Aggregation Function, UDAF
- UDAF 를 생성하려면 기본 클래스인 UserDefinedAggregationFunction을 상속
- UDAF는 현재 스칼라와 자바로만 사용할 수 있음(ver2.3)
  ~~~
  inputSchema : UDAF 입력 파라미터의 스키마를 StructType으로 정의
  bufferSchema : UDAF 중간 결과의 스키마를 StructType으로 정의
  dataType : 반환될 DataType을 정의
  deterministic : UDAF가 동일한 입력값에 대해 항상 동일한 결과를 반환하는지 불리언값으로 정의
  initialize : 집계용 버퍼의 값을 초기화하는 로직을 정의
  update : 입력받은 로우를 기반으로 내부 버퍼를 업데이트하는 로직을 정의
  merge : 두 개의 집계용 버퍼를 병합하는 로직을 정의
  evaluate : 집계의 최종 결과를 생성하는 로직을 정의
  ~~~